## NB2: The multispecies coalescent in space and time


This notebook should be run after first completing notebook 1 (nb1) to simulate linked and unlinked genealogies under a range of species tree scenarios and to generate and save dataframes of statistics which will be loaded and used in this notebook. 

In [1]:
# conda create -n ipcoal python=3.7
# conda activate ipcoal
# conda install ipcoal -c conda-forge -c bioconda

In [2]:
import ipcoal
import toytree
import toyplot, toyplot.pdf, toyplot.svg
import numpy as np
import pandas as pd
import os
from scipy.optimize import curve_fit

In [3]:
# print versions of our libraries 
print('toytree', toytree.__version__)
print('ipcoal', ipcoal.__version__)

toytree 1.1.2
ipcoal 0.1.0


In [49]:
# make a subdirectory for storing some working files
WORKDIR = "results"
os.makedirs(WORKDIR, exist_ok=True)

In [50]:
# Paramaters
coalunits = [0.2, 1.0, 2.0]
treesizes = [10, 50, 100]
treeheights = [100000, 1000000, 10000000]

### A function to estimate exponential decay rate
Depending on the tree shape, the decay is not expected to be exponential. However, in this case that all internodes are exactly the same in all parameters, I believe it is exponential. 

In [15]:
def exponential(x, a, k, b):
    return a * np.exp(x * k) + b


def fit_curve(ntips, coal, th):
    """
    Load the linked and unlinked genealogy databases. 
    Calculate RF dist between trees in each region, and
    the phylogenetic decay rate. Fit an exponential 
    decay function to the decay.
    """
    
    # load the linked tree table
    table = pd.read_csv(
        os.path.join(
            WORKDIR,
            "tree-{}-coal-{}-th-{}-decay.csv"
            .format(ntips, coal, th),
        ), index_col=0,
    )
    
    # load the unlinked tree table
    unlinked = pd.read_csv(
        os.path.join(WORKDIR, "Table-1-full.csv"), 
        index_col=0,
    )
    
    # select unlinked results for this set
    mask = (
        (unlinked.ntips==ntips) & \
        (unlinked.tc==coal) & \
        (unlinked.height==th)
    )

    # calculate phylogenetic linkage
    table["uRF-mean"] = unlinked.loc[mask, "uRF-mean"].values[0]
    table["phylo-linkage"] = 1 - (
        table["lRF-mean"] / table["uRF-mean"]) 
    
    # fit exponential curve
    popt, pcov = curve_fit(
        exponential,
        table["dists"].astype(np.float128), 
        table["phylo-linkage"].astype(np.float128),
        p0=[0.01, -0.005, 1.0]
    )
    return table, popt

### Function to plot decay curves

In [16]:
def plot_phylo_linkage(treesizes, coals, treeheights):
    """
    Draw a plot with selected parameter ranges from
    decay table results dataframes. Loads dataframes.
    """
    # setup canvas
    canvas = toyplot.Canvas(width=350, height=300)
    axes = canvas.cartesian()
    
    # markers for each set
    dashset = {0: "0, 0", 1: "2, 2", 2: "4, 4"}
    markset = {0: 'o', 1: 's', 2: '^'}
    
    # select datasets
    for ntips in treesizes:
        for dash, coal in enumerate(coals):
            for th in treeheights:

                # fit exponential curve
                nt, popt = fit_curve(ntips, coal, th)
                
                # add markers to plot
                axes.scatterplot(
                    nt["dists"],
                    nt["phylo-linkage"], 
                    opacity=0.7,
                    size=6,
                    marker=markset[dash],
                )
                axes.plot(
                    nt["dists"],
                    [exponential(i, popt[0], popt[1], popt[2])
                     for i in nt['dists']
                    ],
                    style={"stroke-dasharray": dashset[dash]},
                )
                print(ntips, coal, th, int(np.log(2.0) / abs(popt[1])), popt[1])
    
    # styling
    axes.x.ticks.show = True
    axes.y.ticks.show = True
    axes.y.label.text = "Phylogenetic linkage"
    axes.x.label.text = "Genomic distance (bp)"
    return canvas

### Estimated and plotted decay curves

In [17]:
for th in treeheights:
    for ntips in treesizes:
        canvas = plot_phylo_linkage([ntips], coalunits, [th])
        
        #save figs
#         toyplot.svg.render(
#             canvas, 
#             "../manuscript/figures/RF-decay-th{}-nt{}.svg"
#             .format(th, ntips)
#         )

/home/deren/miniconda3/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


10 0.2 100000 6576 -0.00010539949879298096
10 1.0 100000 16715 -4.1467219498421746e-05
10 2.0 100000 22627 -3.06327779173262e-05
50 0.2 100000 15890 -4.362029174898527e-05
50 1.0 100000 84731 -8.18046537813126e-06
50 2.0 100000 124415 -5.571209824729164e-06
100 0.2 100000 26873 -2.579336607636519e-05
100 1.0 100000 178444 -3.88438877545161e-06
100 2.0 100000 218558 -3.1714544654570958e-06
10 0.2 1000000 607 -0.0011413442903443862
10 1.0 1000000 2359 -0.000293769763709306
10 2.0 1000000 4659 -0.0001487458455588093
50 0.2 1000000 1838 -0.0003769592710765304
50 1.0 1000000 11884 -5.8325391119607704e-05
50 2.0 1000000 23019 -3.0111235537406372e-05
100 0.2 1000000 3128 -0.00022154192257873453
100 1.0 1000000 22688 -3.0550181398885534e-05
100 2.0 1000000 39616 -1.7496359799680155e-05
10 0.2 10000000 67 -0.010332377974176259
10 1.0 10000000 270 -0.002560871284540722
10 2.0 10000000 443 -0.001561308287190498
50 0.2 10000000 185 -0.0037301266523976067
50 1.0 10000000 1233 -0.0005620710748198672

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t0c6e5901c2b141719626ad6a2be4e276" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 51.82438195425108 L 50.001 51.84239578199854 L 50.001 51.84239578199854 L 50.00150000000001 51.860407711197624 L 50.002 51.8784177420484 L 50.003 51.914432109505434 L 50.004000000000005 51.95043888597011 L 50.005 51.98643807304235 L 50.0065 52.040422627038964 L 50.0085 52.11237548442173 L 50.011500000000005 52.220247908016006 L 50.014500000000005 52.328052134979906 L 50.019 52.48963070104766 L 50.0245 52.68690758032844 L 50.032000000000004 52.955553170835984 L 50.041 53.27736774222139 L 50.053000000000004 53.70550520298658 L 50.068999999999996 54.2746730678116 L 50.089 54.98343865255719 L 50.114999999999995 55.90037792178868 L 50.1485 57.07443392160616 L 50.192 58.58663398157713 L 50.2485 60.53016754538454 L 50.321 62.99041257569817 L 50.4145 66.10826975002593 L 50.535500000000006 70.05298906052624 L 50.692 75.00806822308226 L 50.894 81.16679036590291 L 51.15500000000001 88.74572491744308 L 51.492 97.93457125335412 L 51.9275 108.88219052720463 L 52.49 121.61350009826955 L 53.217 135.98259329567094 L 54.156000000000006 151.5631944993105 L 55.3685 167.61742336130948 L 56.9355 183.12478280603537 L 58.959500000000006 196.88482698721683 L 61.574 207.84120253592442 L 64.952 215.4320288762356 L 69.315 219.8288418809132 L 74.952 221.8551861552421 L 82.23349999999999 222.55200342066277 L 91.64 222.71703573746484 L 103.792 222.741349932077 L 119.49 222.7433090962206 L 139.76850000000002 222.743382382195 L 165.96550000000002 222.74338341233144 L 199.807 222.74338341646177 L 243.5245 222.7433834164651 L 299.9995 222.7433834164651" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.69824263009093 L 50.001 50.70599466117325 L 50.001 50.70599466117325 L 50.00150000000001 50.71374637080704 L 50.002 50.72149775900563 L 50.003 50.736999571150676 L 50.004000000000005 50.75250009771488 L 50.005 50.767999338804955 L 50.0065 50.79124579040912 L 50.0085 50.82223656104998 L 50.011500000000005 50.8687130798189 L 50.014500000000005 50.91517803651422 L 50.019 50.98485379896083 L 50.0245 51.069977758558615 L 50.032000000000004 51.18599332832734 L 50.041 51.325116790922955 L 50.053000000000004 51.51045328094064 L 50.068999999999996 51.757281854205765 L 50.089 52.0653573388596 L 50.114999999999995 52.46509228165846 L 50.1485 52.97886624751913 L 50.192 53.64387896043166 L 50.2485 54.504057101270845 L 50.321 55.60193756868148 L 50.4145 57.008113322483034 L 50.535500000000006 58.81175877728555 L 50.692 61.11788925880144 L 50.894 64.05058300420592 L 51.15500000000001 67.76782719075376 L 51.492 72.44999277315449 L 51.9275 78.31005943667469 L 52.49 85.57245219196645 L 53.217 94.47051426156412 L 54.156000000000006 105.19725045523795 L 55.3685 117.86824353162572 L 56.9355 132.46527488022198 L 58.959500000000006 148.71851561170618 L 61.574 166.05352686098905 L 64.952 183.54575559611564 L 69.315 199.97095064313396 L 74.952 214.03999441572543 L 82.23349999999999 224.740880431098 L 91.64 231.73078686942907 L 103.792 235.4866819207684 L 119.49 237.058428523736 L 139.76850000000002 237.5364542123056 L 165.96550000000002 237.63327138211932 L 199.807 237.64496193049877 L 243.5245 237.64569352643224 L 299.9995 237.64571335743136" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.0 L 50.001 50.00363739614373 L 50.001 50.00363739614373 L 50.00150000000001 50.00727468086568 L 50.002 50.01

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tc81d7162374e416595363c82fcf04e44" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 51.71717688592634 L 50.001 51.72550259478514 L 50.001 51.72550259478514 L 50.00150000000001 51.73382794048202 L 50.002 51.74215292303283 L 50.003 51.75880179875947 L 50.004000000000005 51.77544922209183 L 50.005 51.79209519315668 L 50.0065 51.81706142702931 L 50.0085 51.85034465710537 L 50.011500000000005 51.900258614804315 L 50.014500000000005 51.950159510644276 L 50.019 52.024986370895455 L 50.0245 52.11640153834265 L 50.032000000000004 52.24098791742986 L 50.041 52.39038400923539 L 50.053000000000004 52.58939641820133 L 50.068999999999996 52.85442241417633 L 50.089 53.18518510948984 L 50.114999999999995 53.61431472366629 L 50.1485 54.16579862038573 L 50.192 54.87950355507771 L 50.2485 55.80246522260346 L 50.321 56.980151798191116 L 50.4145 58.48800511743721 L 50.535500000000006 60.421175661110624 L 50.692 62.89144053492068 L 50.894 66.03044081431531 L 51.15500000000001 70.00522138588232 L 51.492 75.00533955212398 L 51.9275 81.25296126684466 L 52.49 88.97906232877297 L 53.217 98.4190869399508 L 54.156000000000006 109.75854126549598 L 55.3685 123.09164361262346 L 56.9355 138.3602408724936 L 58.959500000000006 155.23114992710595 L 61.574 173.04761771633582 L 64.952 190.79767554347714 L 69.315 207.1932841477564 L 74.952 220.94316142725955 L 82.23349999999999 231.12094401564303 L 91.64 237.54135374303013 L 103.792 240.84055909370946 L 119.49 242.14458773265375 L 139.76850000000002 242.5133453368927 L 165.96550000000002 242.58142385917063 L 199.807 242.58873116324264 L 243.5245 242.58912486703852 L 299.9995 242.58913368412462" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.81719106143954 L 50.001 50.818531334259305 L 50.001 50.818531334259305 L 50.00150000000001 50.819871596115135 L 50.002 50.821211847006964 L 50.003 50.8238923158993 L 50.004000000000005 50.82657274093698 L 50.005 50.82925312212079 L 50.0065 50.83327361167207 L 50.0085 50.838634110925106 L 50.011500000000005 50.84667453092657 L 50.014500000000005 50.85471455629145 L 50.019 50.8667738544375 L 50.0245 50.881511790936585 L 50.032000000000004 50.901606839934836 L 50.041 50.92571764409932 L 50.053000000000004 50.95785986082946 L 50.068999999999996 51.00070633461782 L 50.089 51.054248656577194 L 50.114999999999995 51.123827487799645 L 50.1485 51.21343351092223 L 50.192 51.329714333768806 L 50.2485 51.4806222493355 L 50.321 51.67406091459197 L 50.4145 51.92319156090779 L 50.535500000000006 52.24503071784677 L 50.692 52.66034992700135 L 50.894 53.19484717182033 L 51.15500000000001 53.88284937133024 L 51.492 54.766854749499025 L 51.9275 55.90204603775892 L 52.49 57.35636077632176 L 53.217 59.21626281478254 L 54.156000000000006 61.58602144093521 L 55.3685 64.59264750095622 L 56.9355 68.39101187221588 L 58.959500000000006 73.15519259401755 L 61.574 79.08035542608035 L 64.952 86.36995561900383 L 69.315 95.20816972195084 L 74.952 105.7317067506255 L 82.23349999999999 117.9649463727211 L 91.64 131.75711098705486 L 103.792 146.70387343721458 L 119.49 162.0952717210056 L 139.76850000000002 176.93588339990157 L 165.96550000000002 190.08454976540406 L 199.807 200.531589073687 L 243.5245 207.748276197622 L 299.9995 211.9141289770452" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.0 L 50.001 50.00058617961578 L 50.001 50.00058617961578 L 50.00150000000001 50.001172355965885 L 50.002 50.00

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tb9b9edcb56c84d2bb3c8109e503e4512" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 51.42423771796025 L 50.001 51.4290922470138 L 50.001 51.4290922470138 L 50.00150000000001 51.4339466508543 L 50.002 51.438800929485026 L 50.003 51.44850911112995 L 50.004000000000005 51.45821679197444 L 50.005 51.46792397204429 L 50.0065 51.482483803253224 L 50.0085 51.50189515906172 L 50.011500000000005 51.53100843790115 L 50.014500000000005 51.56011721151248 L 50.019 51.60377192615179 L 50.0245 51.65711392807548 L 50.032000000000004 51.729828635487294 L 50.041 51.817049156492146 L 50.053000000000004 51.93328021046446 L 50.068999999999996 52.08814306645992 L 50.089 52.28154197342558 L 50.114999999999995 52.532662430219304 L 50.1485 52.85572530936924 L 50.192 53.27439256788984 L 50.2485 53.81677805508427 L 50.321 54.51044767000845 L 50.4145 55.40122042511784 L 50.535500000000006 56.54762565272069 L 50.692 58.01979941029385 L 50.894 59.90249886957575 L 51.15500000000001 62.306224885161086 L 51.492 65.3623969371672 L 51.9275 69.23395914658308 L 52.49 74.10752905613556 L 53.217 80.2004343504292 L 54.156000000000006 87.73937325532444 L 55.3685 96.94924941465305 L 56.9355 108.02958102366134 L 58.959500000000006 121.077678841969 L 61.574 136.03649140666624 L 64.952 152.60413989583375 L 69.315 170.14453365773846 L 74.952 187.67913152021768 L 82.23349999999999 203.94876605442494 L 91.64 217.66862948819852 L 103.792 227.89940246304684 L 119.49 234.4133803365424 L 139.76850000000002 237.80068548709716 L 165.96550000000002 239.1602193159478 L 199.807 239.55222820622635 L 243.5245 239.62641073135617 L 299.9995 239.63462789443003" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.72346373173578 L 50.001 50.724151548626466 L 50.001 50.724151548626466 L 50.00150000000001 50.72483936284537 L 50.002 50.72552717439258 L 50.003 50.72690278947185 L 50.004000000000005 50.728278393864294 L 50.005 50.72965398756999 L 50.0065 50.731717358091146 L 50.0085 50.734468481383274 L 50.011500000000005 50.738595086174044 L 50.014500000000005 50.7427215947899 L 50.019 50.74891117739064 L 50.0245 50.75647592894166 L 50.032000000000004 50.76679097834644 L 50.041 50.779168244318655 L 50.053000000000004 50.79566991948532 L 50.068999999999996 50.817669760150764 L 50.089 50.84516571574433 L 50.114999999999995 50.8809040716735 L 50.1485 50.926940927467875 L 50.192 50.986702251229275 L 50.2485 51.06429313602552 L 50.321 51.163806783874556 L 50.4145 51.29206234872217 L 50.535500000000006 51.45790190565636 L 50.692 51.672165791706995 L 50.894 51.94833880879308 L 51.15500000000001 52.30453506108033 L 51.492 52.76338426405638 L 51.9275 53.354571624091605 L 52.49 54.11520617912422 L 53.217 55.09337275968959 L 54.156000000000006 56.34863336227218 L 55.3685 57.95602306949181 L 56.9355 60.011064486904004 L 58.959500000000006 62.62868074130658 L 61.574 65.94962896232082 L 64.952 70.1417029661066 L 69.315 75.39587165255719 L 74.952 81.9258397731703 L 82.23349999999999 89.94841830506519 L 91.64 99.66263478792321 L 103.792 111.20576515744887 L 119.49 124.59161419707411 L 139.76850000000002 139.63432525868737 L 165.96550000000002 155.8688425923715 L 199.807 172.49742216747345 L 243.5245 188.42158226639208 L 299.9995 202.40548085750544" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.0 L 50.001 50.00072504259523 L 50.001 50.00072504259523 L 50.00150000000001 50.001450082891076 L 50.002 50.00

...

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t38b25d750f43474d8c15065b887e370e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 52.422286193605025 L 50.001 52.48638992823158 L 50.001 52.48638992823158 L 50.00150000000001 52.55046950291498 L 50.002 52.61452492676082 L 50.003 52.742563358345194 L 50.004000000000005 52.87050529576081 L 50.005 52.99835081172899 L 50.0065 53.189938454410445 L 50.0085 53.4450518473482 L 50.011500000000005 53.82700140607641 L 50.014500000000005 54.20808806444386 L 50.019 54.77810437706721 L 50.0245 55.47216986470014 L 50.032000000000004 56.41399666340499 L 50.041 57.53718169560891 L 50.053000000000004 59.02295569467138 L 50.068999999999996 60.983193339627405 L 50.089 63.40047140609677 L 50.114999999999995 66.48893211925298 L 50.1485 70.38007614478371 L 50.192 75.28829481376236 L 50.2485 81.42768635426003 L 50.321 88.93211023343575 L 50.4145 98.02392821600594 L 50.535500000000006 108.87740985440124 L 50.692 121.5238839399093 L 50.894 135.78927544137682 L 51.15500000000001 151.27941420882865 L 51.492 167.2607241999055 L 51.9275 182.7234174068368 L 52.49 196.474475585012 L 53.217 207.4598618423303 L 54.156000000000006 215.0949161471615 L 55.3685 219.53702618605035 L 56.9355 221.59714371351131 L 58.959500000000006 222.3108576178457 L 61.574 222.48152186399244 L 64.952 222.5069794243138 L 69.315 222.50906253152232 L 74.952 222.5091420421606 L 82.23349999999999 222.5091431881673 L 91.64 222.5091431929151 L 103.792 222.50914319291905 L 119.49 222.50914319291905 L 139.76850000000002 222.50914319291905 L 165.96550000000002 222.50914319291905 L 199.807 222.50914319291905 L 243.5245 222.50914319291905 L 299.9995 222.50914319291905" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.56194724949794 L 50.001 51.57298967622465 L 50.001 51.57298967622465 L 50.00150000000001 51.58403145891628 L 50.002 51.595072597610425 L 50.003 51.61715294315636 L 50.004000000000005 51.639230713162966 L 50.005 51.66130590793058 L 50.0065 51.69441387216618 L 50.0085 51.73854881394728 L 50.011500000000005 51.804731923213375 L 50.014500000000005 51.87089187559724 L 50.019 51.97008840274198 L 50.0245 52.091257910069785 L 50.032000000000004 52.25636382311851 L 50.041 52.4543003419075 L 50.053000000000004 52.71789265529592 L 50.068999999999996 53.068775596524944 L 50.089 53.50645933266174 L 50.114999999999995 54.07392374559945 L 50.1485 54.80254702283638 L 50.192 55.74443063471897 L 50.2485 56.96068303948006 L 50.321 58.509664279289055 L 50.4145 60.48807189407479 L 50.535500000000006 63.01654079873376 L 50.692 66.23434659407752 L 50.894 70.30177508863706 L 51.15500000000001 75.41727189462523 L 51.492 81.79609891887895 L 51.9275 89.67653206487618 L 52.49 99.28017333270304 L 53.217 110.79444561279567 L 54.156000000000006 124.29197219974391 L 55.3685 139.67057410849662 L 56.9355 156.58046418500086 L 58.959500000000006 174.31111339734662 L 61.574 191.81285813714052 L 64.952 207.79681188866647 L 69.315 220.99755100276812 L 74.952 230.58440722888514 L 82.23349999999999 236.48369837407998 L 91.64 239.42059352220912 L 103.792 240.53545647721796 L 119.49 240.83486432300418 L 139.76850000000002 240.88662520080246 L 165.96550000000002 240.89173681355948 L 199.807 240.89198448559043 L 243.5245 240.89198933006858 L 299.9995 240.89198935975392" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.21811198296646 L 50.001 51.22335627167143 L 50.001 51.22335627167143 L 50.00150000000001 51.228600

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tfccf241b00f54c18bf19393e2121c94d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 52.08561609489948 L 50.001 52.12564411825065 L 50.001 52.12564411825065 L 50.00150000000001 52.16566327469874 L 50.002 52.205673566208 L 50.003 52.28566756226449 L 50.004000000000005 52.365626122124766 L 50.005 52.445549261486676 L 50.0065 52.56536759141652 L 50.0085 52.72500153930075 L 50.011500000000005 52.96418737567337 L 50.014500000000005 53.20305548512203 L 50.019 53.560762834412 L 50.0245 53.9969932369907 L 50.032000000000004 54.59014224520833 L 50.041 55.29932408238605 L 50.053000000000004 56.24051138958808 L 50.068999999999996 57.487668571518036 L 50.089 59.0342329284607 L 50.114999999999995 61.024385929243664 L 50.1485 63.55505146912756 L 50.192 66.78557884214769 L 50.2485 70.8896585890151 L 50.321 76.00766729323314 L 50.4145 82.36998588286066 L 50.535500000000006 90.2218383472701 L 50.692 99.77256887835128 L 50.894 111.16033888669796 L 51.15500000000001 124.44276995959487 L 51.492 139.47045702045517 L 51.9275 155.84597089712702 L 52.49 172.8189158705316 L 53.217 189.33294777349604 L 54.156000000000006 204.1220875425418 L 55.3685 216.0355001181603 L 56.9355 224.41993158611814 L 58.959500000000006 229.37303812363234 L 61.574 231.7136327019253 L 64.952 232.54505416826788 L 69.315 232.7501612262707 L 74.952 232.7820032422032 L 82.23349999999999 232.78474464427032 L 91.64 232.78485623261363 L 103.792 232.7848579797122 L 119.49 232.78485798775677 L 139.76850000000002 232.78485798776444 L 165.96550000000002 232.78485798776444 L 199.807 232.78485798776444 L 243.5245 232.78485798776444 L 299.9995 232.78485798776444" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.30174347342224 L 50.001 51.3074517764788 L 50.001 51.3074517764788 L 50.00150000000001 51.31315990514824 L 50.002 51.31886785943607 L 50.003 51.33028324488775 L 50.004000000000005 51.341697932876684 L 50.005 51.353111923445205 L 50.0065 51.370231601728314 L 50.0085 51.393055398865854 L 50.011500000000005 51.427285865464526 L 50.014500000000005 51.461510058156534 L 50.019 51.512834586134645 L 50.0245 51.575545403676884 L 50.032000000000004 51.661026199574714 L 50.041 51.763551461291165 L 50.053000000000004 51.90016413908181 L 50.068999999999996 52.0821586336911 L 50.089 52.30940169616027 L 50.114999999999995 52.60440283106247 L 50.1485 52.98381018405338 L 50.192 53.47531596087063 L 50.2485 54.111761325482455 L 50.321 54.925227095591424 L 50.4145 55.96901118385016 L 50.535500000000006 57.310969086594426 L 50.692 59.03199227414977 L 50.894 61.22918601800467 L 51.15500000000001 64.02827275969646 L 51.492 67.57701692393847 L 51.9275 72.05609604163996 L 52.49 77.66774407387315 L 53.217 84.64055423575789 L 54.156000000000006 93.2002586627197 L 55.3685 103.5509427618291 L 56.9355 115.84075021409112 L 58.959500000000006 130.06904490421857 L 61.574 146.02685851454555 L 64.952 163.20776815915252 L 69.315 180.7457852665148 L 74.952 197.4730472438294 L 82.23349999999999 212.08234474467554 L 91.64 223.47983108368177 L 103.792 231.17051419292554 L 119.49 235.47821068544934 L 139.76850000000002 237.37937562827148 L 165.96550000000002 237.99822755143333 L 199.807 238.13486860028576 L 243.5245 238.1532835797952 L 299.9995 238.15460863087768" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 50.59738259008952 L 50.001 50.60064944368383 L 50.001 50.60064944368383 L 50.00150000000001 50.603916240120

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t0dca309e49b84db3a626f5b10cf13492" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 56.243626758173335 L 50.001 58.01187852869664 L 50.001 58.01187852869664 L 50.00150000000001 59.76195411687066 L 50.002 61.494040358987775 L 50.003 64.90498256734057 L 50.004000000000005 68.2461617875349 L 50.005 71.51900486125254 L 50.0065 76.30318874352919 L 50.0085 82.45573080319079 L 50.011500000000005 91.22065192054112 L 50.014500000000005 99.45869850443663 L 50.019 110.89663592784729 L 50.0245 123.50626190029008 L 50.032000000000004 138.5469185652423 L 50.041 153.77337486628534 L 50.053000000000004 170.13365834954922 L 50.068999999999996 186.49992220134686 L 50.089 200.63919612295757 L 50.114999999999995 212.12283547806817 L 50.1485 220.18744711482375 L 50.192 224.9780863632305 L 50.2485 227.24320898112904 L 50.321 228.03755581172985 L 50.4145 228.23311540374 L 50.535500000000006 228.26351899351033 L 50.692 228.26612949257773 L 50.894 228.26623491252684 L 51.15500000000001 228.26623655240286 L 51.492 228.26623655988473 L 51.9275 228.26623655989184 L 52.49 228.26623655989184 L 53.217 228.26623655989184 L 54.156000000000006 228.26623655989184 L 55.3685 228.26623655989184 L 56.9355 228.26623655989184 L 58.959500000000006 228.26623655989184 L 61.574 228.26623655989184 L 64.952 228.26623655989184 L 69.315 228.26623655989184 L 74.952 228.26623655989184 L 82.23349999999999 228.26623655989184 L 91.64 228.26623655989184 L 103.792 228.26623655989184 L 119.49 228.26623655989184 L 139.76850000000002 228.26623655989184 L 165.96550000000002 228.26623655989184 L 199.807 228.26623655989184 L 243.5245 228.26623655989184 L 299.9995 228.26623655989184" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 53.49871183440911 L 50.001 53.93287822680812 L 50.001 53.93287822680812 L 50.00150000000001 54.36593419739057 L 50.002 54.797882586165954 L 50.003 55.65846794203317 L 50.004000000000005 56.514656869546755 L 50.005 57.366471828515174 L 50.0065 58.63604192455472 L 50.0085 60.313700761072255 L 50.011500000000005 62.79818599446796 L 50.014500000000005 65.2447883314939 L 50.019 68.8449225605116 L 50.0245 73.13388154787195 L 50.032000000000004 78.79112110527389 L 50.041 85.2991087724594 L 50.053000000000004 93.52266647589292 L 50.068999999999996 103.7299650307433 L 50.089 115.36723137986354 L 50.114999999999995 128.8185662809571 L 50.1485 143.70786836091122 L 50.192 159.59504762025276 L 50.2485 175.59081832412036 L 50.321 190.3751759141367 L 50.4145 202.88655515399563 L 50.535500000000006 212.29469701445328 L 50.692 218.33754663817845 L 50.894 221.50711250851245 L 51.15500000000001 222.79542672230505 L 51.492 223.17273444313133 L 51.9275 223.24565266654105 L 52.49 223.25394062938824 L 53.217 223.25442113908005 L 54.156000000000006 223.25443293317764 L 55.3685 223.2544330299357 L 56.9355 223.25443303013043 L 58.959500000000006 223.2544330301305 L 61.574 223.2544330301305 L 64.952 223.2544330301305 L 69.315 223.2544330301305 L 74.952 223.2544330301305 L 82.23349999999999 223.2544330301305 L 91.64 223.2544330301305 L 103.792 223.2544330301305 L 119.49 223.2544330301305 L 139.76850000000002 223.2544330301305 L 165.96550000000002 223.2544330301305 L 199.807 223.2544330301305 L 243.5245 223.2544330301305 L 299.9995 223.2544330301305" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 52.99701428527646 L 50.001 53.28021431388479 L 50.001 53.28021431388479 L 50.0015000000000

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t2378edf278554a7aaafdc430a3dea7c4" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 55.267191991925976 L 50.001 55.94263276019704 L 50.001 55.94263276019704 L 50.00150000000001 56.615558742008076 L 50.002 57.285979300357745 L 50.003 58.619341424496255 L 50.004000000000005 59.942793341731516 L 50.005 61.25640870962373 L 50.0065 63.20854801109521 L 50.0085 65.77765030592585 L 50.011500000000005 69.56014061983092 L 50.014500000000005 73.25891621994869 L 50.019 78.65415626830206 L 50.0245 85.00704552484987 L 50.032000000000004 93.26051193631216 L 50.041 102.57401361908533 L 50.053000000000004 114.05808105094451 L 50.068999999999996 127.8551867355815 L 50.089 142.9394443773734 L 50.114999999999995 159.46767417582424 L 50.1485 176.5425846275991 L 50.192 193.20888979818233 L 50.2485 208.16116750587344 L 50.321 220.07608210618736 L 50.4145 228.4155277493629 L 50.535500000000006 233.33030628606258 L 50.692 235.63937168424263 L 50.894 236.45119985839992 L 51.15500000000001 236.64931701415944 L 51.492 236.67962107906962 L 51.9275 236.68218615101333 L 52.49 236.68228817684516 L 53.217 236.68228972889534 L 54.156000000000006 236.68228973576586 L 55.3685 236.68228973577212 L 56.9355 236.68228973577212 L 58.959500000000006 236.68228973577212 L 61.574 236.68228973577212 L 64.952 236.68228973577212 L 69.315 236.68228973577212 L 74.952 236.68228973577212 L 82.23349999999999 236.68228973577212 L 91.64 236.68228973577212 L 103.792 236.68228973577212 L 119.49 236.68228973577212 L 139.76850000000002 236.68228973577212 L 165.96550000000002 236.68228973577212 L 199.807 236.68228973577212 L 243.5245 236.68228973577212 L 299.9995 236.68228973577212" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 52.15086673834644 L 50.001 52.255192893940446 L 50.001 52.255192893940446 L 50.00150000000001 52.35946042729642 L 50.002 52.463669371355095 L 50.003 52.6719116232504 L 50.004000000000005 52.87991991278114 L 50.005 53.087694502806414 L 50.0065 53.39891877554957 L 50.0085 53.813068972394774 L 50.011500000000005 54.43255101785884 L 50.014500000000005 55.04994742449669 L 50.019 55.972146815411584 L 50.0245 57.09296247929885 L 50.032000000000004 58.610222461582964 L 50.041 60.41412925223231 L 50.053000000000004 62.79112321667249 L 50.068999999999996 65.91097853620882 L 50.089 69.73270855097533 L 50.114999999999995 74.57421891584004 L 50.1485 80.6072882939193 L 50.192 88.10975955644635 L 50.2485 97.32237240575898 L 50.321 108.31825540818895 L 50.4145 121.23824290968948 L 50.535500000000006 136.06377611627434 L 50.692 152.47819495399042 L 50.894 169.81319603825085 L 51.15500000000001 187.10405789661337 L 51.492 203.0948109053904 L 51.9275 216.53455577115636 L 52.49 226.5067849403992 L 53.217 232.81984073769001 L 54.156000000000006 236.07559852606673 L 55.3685 237.36863855334374 L 56.9355 237.73690686980092 L 58.959500000000006 237.80543996333995 L 61.574 237.8128744496826 L 64.952 237.81328050768795 L 69.315 237.81328975552347 L 74.952 237.81328982445964 L 82.23349999999999 237.81328982458183 L 91.64 237.8132898245819 L 103.792 237.8132898245819 L 119.49 237.8132898245819 L 139.76850000000002 237.8132898245819 L 165.96550000000002 237.8132898245819 L 199.807 237.8132898245819 L 243.5245 237.8132898245819 L 299.9995 237.8132898245819" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.39847333726577 L 50.001 51.46298713734287 L 50.001 51.46298713734287 L 50.001500

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t623124f736664f68977c4f62f55c56f8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 300.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 50.0005 54.10766087257541 L 50.001 54.477364280275744 L 50.001 54.477364280275744 L 50.00150000000001 54.84632738246988 L 50.002 55.21455166156839 L 50.003 55.94878966528481 L 50.004000000000005 56.68009009144801 L 50.005 57.408464692870865 L 50.0065 58.49556626722525 L 50.0085 59.93490350829882 L 50.011500000000005 62.07238662396675 L 50.014500000000005 64.18431698513585 L 50.019 67.30496833453361 L 50.0245 71.04343204223773 L 50.032000000000004 76.01029970813721 L 50.041 81.77673615492766 L 50.053000000000004 89.14870888866952 L 50.068999999999996 98.44223585960874 L 50.089 109.25144339661279 L 50.114999999999995 122.06806908761672 L 50.1485 136.72959498641907 L 50.192 153.05326049434424 L 50.2485 170.4193897243636 L 50.321 187.64979613450356 L 50.4145 203.61870604935055 L 50.535500000000006 217.11558454949778 L 50.692 227.19055947512493 L 50.894 233.5982803284156 L 51.15500000000001 236.93081221477743 L 51.492 238.26771928180403 L 51.9275 238.65343330953172 L 52.49 238.72641421773972 L 53.217 238.7345010413445 L 54.156000000000006 238.7349540184842 L 55.3685 238.7349646858336 L 56.9355 238.73496476893678 L 58.959500000000006 238.73496476909244 L 61.574 238.73496476909247 L 64.952 238.73496476909247 L 69.315 238.73496476909247 L 74.952 238.73496476909247 L 82.23349999999999 238.73496476909247 L 91.64 238.73496476909247 L 103.792 238.73496476909247 L 119.49 238.73496476909247 L 139.76850000000002 238.73496476909247 L 165.96550000000002 238.73496476909247 L 199.807 238.73496476909247 L 243.5245 238.73496476909247 L 299.9995 238.73496476909247" style="stroke:rgb(40%,76.1%,64.7%);stroke-dasharray:0, 0;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.92499830037336 L 50.001 51.97762615001473 L 50.001 51.97762615001473 L 50.00150000000001 52.03023940452871 L 50.002 52.08283806796279 L 50.003 52.18799163777659 L 50.004000000000005 52.29308689181458 L 50.005 52.39812386241745 L 50.0065 52.555570107581744 L 50.0085 52.76529474397233 L 50.011500000000005 53.07944574318681 L 50.014500000000005 53.39307436922927 L 50.019 53.86253975827808 L 50.0245 54.434741569244 L 50.032000000000004 55.21220861265865 L 50.041 56.140909392127455 L 50.053000000000004 57.37198654861897 L 50.068999999999996 59.0007266902163 L 50.089 61.01642770923237 L 50.114999999999995 63.6036272281945 L 50.1485 66.8825908294052 L 50.192 71.05042127341923 L 50.2485 76.31581243850457 L 50.321 82.83498638083431 L 50.4145 90.8645048803053 L 50.535500000000006 100.65619809072854 L 50.692 112.38394098905849 L 50.894 126.09143209832092 L 51.15500000000001 141.67356773555827 L 51.492 158.7279211035391 L 51.9275 176.53377673703852 L 52.49 193.9994238753506 L 53.217 209.82808622273635 L 54.156000000000006 222.76562032201602 L 55.3685 232.03303711998765 L 56.9355 237.64305736773434 L 58.959500000000006 240.37535888066532 L 61.574 241.38415245528583 L 64.952 241.6457224550518 L 69.315 241.68894658849058 L 74.952 241.6929790077587 L 82.23349999999999 241.693160674573 L 91.64 241.69316391347616 L 103.792 241.6931639310984 L 119.49 241.69316393111927 L 139.76850000000002 241.69316393111927 L 165.96550000000002 241.69316393111927 L 199.807 241.69316393111927 L 243.5245 241.69316393111927 L 299.9995 241.69316393111927" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-dasharray:2, 2;stroke-opacity:1.0;stroke-width:2.0"> <path d="M 50.0005 51.29109114317612 L 50.001 51.32332250273877 L 50.001 51.32332250273877 L 50.00150000000001 51.3

In [32]:
decay_table = pd.DataFrame({
    "ntips": np.repeat(treesizes, len(coalunits) * len(treeheights)),
    "tc": np.tile(np.repeat(coalunits, len(treesizes)), len(treeheights)),
    "height": np.tile(np.tile(treeheights, len(treesizes)), len(coalunits)).astype(int),
    "decay_rate": 0.,
})

for idx in decay_table.index:
    dat = decay_table.loc[idx]
    rate = fit_curve(int(dat.ntips), dat.tc, int(dat.height))[1][1]
    decay_table.loc[idx, "decay_rate"] = rate
    
# add log 
decay_table["log_decay"] = np.log(abs(decay_table["decay_rate"]))

/home/deren/miniconda3/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [46]:
decay_table["half-life"] = (np.log(2) / decay_table.decay_rate * -1).astype(int)

In [68]:
final["half-life"] = 0.

for ntips in treesizes:
    for tc in coalunits:
        for th in treeheights:
            maskf = (
                (final.ntips==ntips) & \
                (final.tc==tc) & \
                (final.height==th)
            )
            maskr = (
                (decay_table.ntips==ntips) & \
                (decay_table.tc==tc) & \
                (decay_table.height==th)
            )
            final.loc[maskf, 'half-life'] = decay_table.loc[maskr, 'half-life']

In [33]:
# save to CSV
decay_table.to_csv(os.path.join(WORKDIR, "decay_table.csv"))

In [71]:
#decay_table = pd.read_csv("results/decay_table.csv", index_col=0)

In [51]:
# reload the unlinked tree table
final = pd.read_csv(
    os.path.join(WORKDIR, "Table-1-full.csv"), 
    index_col=0,
)

In [76]:
(
    final[["ntips", "height", "tc", "tg", "Ne", "block-mean", "uRF-mean", "lRF-5K-mean", "half-life"]]
    .round(2)
    .to_csv(os.path.join(WORKDIR, "Final-table-1.csv"))
)

In [77]:
(
    final[["ntips", "height", "tc", "tg", "Ne", "block-mean", "uRF-mean", "lRF-5K-mean", "half-life"]]
    .round(2)
    .to_latex(os.path.join(WORKDIR, "Final-table-1.latex"))
)